In [ ]:
from ml_carbucks.adapters import YoloUltralyticsAdapter, RtdetrUltralyticsAdapter, EfficientDetAdapter, FasterRcnnAdapter
from ml_carbucks.utils.DatasetsPathManager import DatasetsPathManager
from ml_carbucks.utils.logger import setup_logger

logger = setup_logger(__name__)


## Ensemble saving debugging

In [ ]:
import pickle as pkl

from ml_carbucks import OPTUNA_DIR, PRODUCTS_DIR
from ml_carbucks.adapters.EnsembleModel import EnsembleModel
from ml_carbucks.optmization.TrialParamWrapper import TrialParamWrapper


adapters = [
        YoloUltralyticsAdapter(
            checkpoint=PRODUCTS_DIR / "best_pickled_YoloUltralyticsAdapter_model.pkl"
        ),
        RtdetrUltralyticsAdapter(
            checkpoint=PRODUCTS_DIR / "best_pickled_RtdetrUltralyticsAdapter_model.pkl"
        ),
        FasterRcnnAdapter(
            checkpoint=PRODUCTS_DIR / "best_pickled_FasterRcnnAdapter_model.pkl"
        ),
        EfficientDetAdapter(
            checkpoint=PRODUCTS_DIR / "best_pickled_EfficientDetAdapter_model.pkl"
        ),
    ]

trial_params = {
    "fusion_strategy": "nms",
    "fusion_conf_threshold": 0.1007298895150254,
    "fusion_iou_threshold": 0.3961959035309548,
    "fusion_max_detections": 5,
    "fusion_norm_method": "quantile",
    "fusion_trust_factor_0": 2.9826045621475172,
    "fusion_trust_factor_1": 2.9578690014499065,
    "fusion_trust_factor_2": 2.277096902306202,
    "fusion_trust_factor_3": 2.8984911425609545
}

ensemble_adapters = [adapter for adapter in adapters]
ensemble_params = TrialParamWrapper.convert_ensemble_params_to_model_format(
    trial_params, ensemble_size=len(ensemble_adapters)
) 
obj = pkl.load(open(OPTUNA_DIR / "ensemble" / "prestep_a4605cd505acf557.pkl", "rb"))
(
    adapters_predictions,
    ground_truths,
    distributions,
    adapters_crossval_metrics,
    adapters_dataset_metrics,
) = obj


ensemble_model = EnsembleModel(
    **ensemble_params,
    adapters=ensemble_adapters,
    distributions=distributions,
)


ensemble_model.save(PRODUCTS_DIR / "ensemble" , suffix="debugging")

## Ensemble pickled model evalaution vs trial model pickled evaluation

In [ ]:
emodel = EfficientDetAdapter(checkpoint=PRODUCTS_DIR / "ensemble" / "adapter_3_modeldebugging.pkl")
eres = emodel.evaluate(DatasetsPathManager.CARBUCKS_VAL_STANDARD)
logger.info(eres["map_50"])

In [ ]:
emodel2 = EfficientDetAdapter(checkpoint=PRODUCTS_DIR / "best_pickled_EfficientDetAdapter_model.pkl")
eres2 = emodel2.evaluate(DatasetsPathManager.CARBUCKS_VAL_STANDARD)
logger.info(eres2["map_50"])

## Ensemble pickled evaluation 

In [ ]:
loaded_ensemble = EnsembleModel(checkpoint=PRODUCTS_DIR / "ensemble" / "ensemble_modeldebugging.pkl")

lres = loaded_ensemble.evaluate(DatasetsPathManager.CARBUCKS_VAL_STANDARD)

logger.info(lres["map_50"])

In [ ]:
from ml_carbucks.utils.postprocessing import plot_pr_curves_with_ap50
plot_pr_curves_with_ap50(
    lres,
    save=PRODUCTS_DIR / "ensemble" / "debugging_pr_curves.png"
)

## Ensemble optimizatin results debugging

In [ ]:
from ml_carbucks.adapters import EnsembleModel
from ml_carbucks import OPTUNA_DIR
from ml_carbucks.utils.DatasetsPathManager import DatasetsPathManager

opt_ensemble = EnsembleModel(checkpoint=OPTUNA_DIR / "ensemble" / "20251201_114723_demo_combined_explorative" / "ensemble_model20251201_114723_demo_combined_explorative.pkl")
opt_ensemble.set_params({"verbose": True})
ores = opt_ensemble.evaluate(DatasetsPathManager.CARBUCKS_VAL_STANDARD)

logger.info(ores["map_50"])

In [ ]:
from PIL import Image
import numpy as np
from ml_carbucks.utils.inference import plot_img_pred, plot_img_pred_subplots

from ml_carbucks import DATA_DIR

img_path = DATA_DIR / "final_carbucks"/ "standard"/ "images"/ "val" / "bd3f80e0-c3d0-4e2c-bc9e-0d5de1e50fae.jpg"

# just laod image aIMagefile

img = Image.open(img_path)

import base64
from io import BytesIO

image_base64 = base64.b64encode(open(img_path, "rb").read()).decode('utf-8')

img_np = np.array(img)

pred1 = opt_ensemble.predict([img_np])
pred2 = opt_ensemble.predict_from_base64([image_base64])

from matplotlib import pyplot as plt
import torch 

logger.info(pred1[0]["boxes"])
logger.info(img_np.shape)
plot_img_pred_subplots(
    [torch.tensor(img_np).permute(2,0,1)],
    [pred1[0]["boxes"]],
    labels_list=[pred1[0]["labels"]],
    scores_list=[pred1[0]["scores"]],
    coords="xyxy",
    figsize=(5,5)
)

logger.info(pred1[0]["labels"])
plt.imshow(img_np)
for box in pred1[0]["boxes"]:
    x1, y1, x2, y2 = box
    width = x2 - x1
    height = y2 - y1
    rect = plt.Rectangle((x1, y1), width, height, fill=False, color="red", linewidth=2)
    plt.gca().add_patch(rect)
plt.show()

logger.info(pred2[0]["labels"])
plt.imshow(img_np)
for box in pred2[0]["boxes"]:
    x1, y1, x2, y2 = box
    width = x2 - x1
    height = y2 - y1
    rect = plt.Rectangle((x1, y1), width, height, fill=False, color="red", linewidth=2)
    plt.gca().add_patch(rect)
plt.show()

In [ ]:
from ml_carbucks.adapters import EnsembleModel
from ml_carbucks import OPTUNA_DIR, DATA_DIR
from ml_carbucks.utils.DatasetsPathManager import DatasetsPathManager
from PIL import Image
from ml_carbucks.utils.inference import plot_img_pred_subplots
import numpy as np
from ml_carbucks.utils.logger import setup_logger


logger = setup_logger(__name__)


opt_ensemble = EnsembleModel(checkpoint=OPTUNA_DIR / "ensemble" / "20251201_114723_demo_combined_explorative" / "ensemble_model20251201_114723_demo_combined_explorative.pkl")
opt_ensemble.set_params({"verbose": True})
img_path = DATA_DIR / "final_carbucks"/ "standard"/ "images"/ "val" / "bd3f80e0-c3d0-4e2c-bc9e-0d5de1e50fae.jpg"

# just laod image aIMagefile

img = Image.open(img_path)

import base64
from io import BytesIO
import torch 


img_np = np.array(img)

pred1 = opt_ensemble.adapters[1].predict([img_np])


logger.info(pred1[0]["boxes"])
logger.info(img_np.shape)
plot_img_pred_subplots(
    [torch.tensor(img_np).permute(2,0,1)],
    [pred1[0]["boxes"]],
    labels_list=[pred1[0]["labels"]],
    scores_list=[pred1[0]["scores"]],
    coords="xyxy",
    figsize=(5,5)
)

In [ ]:
from ml_carbucks.adapters import RtdetrUltralyticsAdapter
from ml_carbucks import DATA_DIR
from PIL import Image
import numpy as np
from ml_carbucks.utils.logger import setup_logger
from ml_carbucks.utils.inference import plot_img_pred_subplots
import torch

logger = setup_logger(__name__)

model = RtdetrUltralyticsAdapter(checkpoint="/home/damian/Desktop/Projects/Bachelor/ml/optuna/ensemble/20251201_114723_demo_combined_explorative/adapter_1_model20251201_114723_demo_combined_explorative.pkl")
img_path = DATA_DIR / "final_carbucks"/ "standard"/ "images"/ "val" / "bd3f80e0-c3d0-4e2c-bc9e-0d5de1e50fae.jpg"


img = Image.open(img_path)
img_np = np.array(img)

pred = model.predict([img_np])
logger.info(pred[0])

plot_img_pred_subplots(
    [torch.tensor(img_np).permute(2,0,1)],
    [pred[0]["boxes"]],
    labels_list=[pred[0]["labels"]],
    scores_list=[pred[0]["scores"]],
    coords="xyxy",
    figsize=(5,5)
)
